In [10]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [11]:
cols = ['timestamp','tweet_text','username','all_hashtags']

In [12]:
df = pd.read_csv("FacebookDown_instagramdown.csv",header=None, names=cols)

In [13]:
df = df[df['timestamp']!='timestamp']

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 453338 entries, 1 to 453855
Data columns (total 4 columns):
timestamp       453338 non-null object
tweet_text      453338 non-null object
username        453338 non-null object
all_hashtags    453338 non-null object
dtypes: object(4)
memory usage: 17.3+ MB


In [16]:
tweets = df.copy()

In [17]:
tweets.head()

,timestamp,tweet_text,username,all_hashtags
1,2019-03-14 16:27:07,"True, true. #Instagram #Instagramdown #Facebo...",bklyn_hayshun,"['Instagram', 'Instagramdown', 'Facebookdown']"
2,2019-03-14 16:26:46,How to prepare yourself for social media outag...,Postfity,"['facebookdown', 'instagramdown']"
3,2019-03-14 16:25:41,When #FacebookDown #facebookoutage #InstagramB...,NikeMC,"['FacebookDown', 'facebookoutage', 'InstagramB..."
4,2019-03-14 16:24:50,#FacebookDown and #instagramdown make me conte...,ftchkn,"['FacebookDown', 'instagramdown', 'crypto', 'b..."
5,2019-03-14 16:24:01,"Facebook, Instagram And Whatsapp Hit By Major ...",DailyMulligan,"['FacebookDown', 'instagramdown', 'whatsappdow..."


In [18]:
tweets.describe()

,timestamp,tweet_text,username,all_hashtags
count,453338,453338,453338,453338
unique,5585,6677,5757,2866
top,2019-03-14 14:34:19,"Thank you @Twitter for being you. Yesterday, @...",BestPhotosFilms,"['FacebookDown', 'instagramdown']"
freq,892,890,2674,37958


# Data Preparation

## HTML encoding

In [19]:
pd.options.display.max_colwidth
text_with_html_encoding = tweets[tweets['tweet_text'].str.contains('&amp')]['tweet_text'].iloc[0]
print(text_with_html_encoding)

How to prepare yourself for social media outage? How can you make sure your social media strategy is sustainable? Check out our guest post on GetaNewsletter's blog &amp; find out how to use social media to collect email subscribers https://t.co/yKXmmCwgpA #facebookdown #instagramdown


In [20]:
from bs4 import BeautifulSoup
example1 = BeautifulSoup(text_with_html_encoding, 'lxml')
print(example1.get_text())

How to prepare yourself for social media outage? How can you make sure your social media strategy is sustainable? Check out our guest post on GetaNewsletter's blog & find out how to use social media to collect email subscribers https://t.co/yKXmmCwgpA #facebookdown #instagramdown


## Lower Case

In [21]:
tweets['tweet_text'] = tweets['tweet_text'].astype(str).str.lower()

## '@' mention

##### I noticed that removing the mention of either facebook, instagram or twitter would make me lose sometimes the tweet's meaning. So I chose to remove any other mention. For example:
###### "thank you twitter for being you. yesterday, facebook and twitter were messing"
###### has more sense than 
###### "thank you  for being you. yesterday,  and  were messing"

In [22]:
# The most frequent tweet as I noticed it contained already a '@' from the tweets.describe()
tweets['tweet_text'].mode()[1]

'thank you @twitter for being you. yesterday, @facebook and @instagram were messing with our mojo #facebookdown #instagramdown'

In [23]:
import re
example = tweets['tweet_text'].mode()[1] + '@amine @hosni'
example = re.sub(r'@facebook','facebook',example)
example = re.sub(r'@twitter','twitter',example)
example = re.sub(r'@instagram','twitter',example)
example = re.sub(r'@[A-Za-z0-9]+','',example)
example

'thank you twitter for being you. yesterday, facebook and twitter were messing with our mojo #facebookdown #instagramdown '

## Defining data cleaning function

In [24]:
pattern = '@facebook|@twitter|@instagram|@[a-z0-9]+|https?://[A-Za-z0-9./]+|www.[^ ]+'

In [25]:
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

In [31]:
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(pattern, '', souped)
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], stripped)
    letters_only = re.sub("[^a-z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(letters_only)
    return (" ".join(words)).strip()

testing = tweets['tweet_text'].iloc[:10]
test_result = []
for t in testing:
    test_result.append(tweet_cleaner(t))
test_result

['true true instagram instagramdown facebookdown',
 'how to prepare yourself for social media outage how can you make sure your social media strategy is sustainable check out our guest post on getanewsletter s blog find out how to use social media to collect email subscribers facebookdown instagramdown',
 'when facebookdown facebookoutage instagramblackout instagramdown listening to uptight everything s alright by stevie wonder on',
 'facebookdown and instagramdown make me contemplate life so much teenagers nowadays spend more than hours each day on social network while they can spend that time educating themselves about crypto blockchain and what project can generate like bcnex bigbom zerobank',
 'facebook instagram and whatsapp hit by major outage facebookdown instagramdown whatsappdown worldnews',
 'the blessings of the social blackout for our brain peace focus brain addiction hypothalamus priorities resilience facebookdown instagramdown',
 'checkout this coversong souhilabenlachhab

In [35]:
nums = [0,int(len(tweets)/2),len(tweets)]

In [36]:
nums

[0, 226669, 453338]

In [37]:
%%time
print("Cleaning and parsing the tweets...\n")
clean_tweets = []
for i in range(nums[0],nums[1]):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[1] ))                                                                   
    clean_tweets.append(tweet_cleaner(tweets['tweet_text'].iloc[i]))

Cleaning and parsing the tweets...

Tweets 10000 of 226669 has been processed
Tweets 20000 of 226669 has been processed
Tweets 30000 of 226669 has been processed
Tweets 40000 of 226669 has been processed
Tweets 50000 of 226669 has been processed
Tweets 60000 of 226669 has been processed
Tweets 70000 of 226669 has been processed
Tweets 80000 of 226669 has been processed
Tweets 90000 of 226669 has been processed
Tweets 100000 of 226669 has been processed
Tweets 110000 of 226669 has been processed
Tweets 120000 of 226669 has been processed
Tweets 130000 of 226669 has been processed
Tweets 140000 of 226669 has been processed
Tweets 150000 of 226669 has been processed
Tweets 160000 of 226669 has been processed
Tweets 170000 of 226669 has been processed
Tweets 180000 of 226669 has been processed
Tweets 190000 of 226669 has been processed
Tweets 200000 of 226669 has been processed
Tweets 210000 of 226669 has been processed
Tweets 220000 of 226669 has been processed
Wall time: 1min 8s


In [38]:
len(clean_tweets)

226669

In [39]:
%%time
print("Cleaning and parsing the tweets...\n")
for i in range(nums[1],nums[2]):
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[2] ))                                                                    
    clean_tweets.append(tweet_cleaner(tweets['tweet_text'].iloc[i]))

Cleaning and parsing the tweets...

Tweets 230000 of 453338 has been processed
Tweets 240000 of 453338 has been processed
Tweets 250000 of 453338 has been processed
Tweets 260000 of 453338 has been processed
Tweets 270000 of 453338 has been processed
Tweets 280000 of 453338 has been processed
Tweets 290000 of 453338 has been processed
Tweets 300000 of 453338 has been processed
Tweets 310000 of 453338 has been processed
Tweets 320000 of 453338 has been processed
Tweets 330000 of 453338 has been processed
Tweets 340000 of 453338 has been processed
Tweets 350000 of 453338 has been processed
Tweets 360000 of 453338 has been processed
Tweets 370000 of 453338 has been processed
Tweets 380000 of 453338 has been processed
Tweets 390000 of 453338 has been processed
Tweets 400000 of 453338 has been processed
Tweets 410000 of 453338 has been processed
Tweets 420000 of 453338 has been processed
Tweets 430000 of 453338 has been processed
Tweets 440000 of 453338 has been processed
Tweets 450000 of 4

In [40]:
len(clean_tweets)

453338

In [57]:
pd.set_option('display.max_colwidth', -1)
clean_df = pd.DataFrame(clean_tweets,columns=['tweet'])
clean_df.head()

,tweet
0,true true instagram instagramdown facebookdown
1,how to prepare yourself for social media outage how can you make sure your social media strategy is sustainable check out our guest post on getanewsletter s blog find out how to use social media to collect email subscribers facebookdown instagramdown
2,when facebookdown facebookoutage instagramblackout instagramdown listening to uptight everything s alright by stevie wonder on
3,facebookdown and instagramdown make me contemplate life so much teenagers nowadays spend more than hours each day on social network while they can spend that time educating themselves about crypto blockchain and what project can generate like bcnex bigbom zerobank
4,facebook instagram and whatsapp hit by major outage facebookdown instagramdown whatsappdown worldnews


In [58]:
clean_df.describe()

,tweet
count,453338
unique,6492
top,facebookdown and instagramdown so listen to my motivation video
freq,1784


In [61]:
clean_df.drop_duplicates(subset='tweet',keep='first',inplace=True)
clean_df.describe()

,tweet
count,6492
unique,6492
top,f in chat for instagram and facebook facebookdown instagramdown instagramblackout
freq,1


In [63]:
clean_df.to_csv('clean_unique_tweets.csv',encoding='utf-8',index=False)

In [46]:
clean_df['number_of_retweets'].value_counts()

445     206480
446     88308 
447     11175 
890     5340  
1       5173  
439     4829  
423     4653  
429     4290  
386     4246  
381     3429  
392     3136  
435     3045  
277     2770  
417     2502  
378     2268  
398     1990  
380     1900  
357     1785  
1784    1784  
891     1782  
1780    1780  
427     1708  
411     1644  
409     1636  
265     1590  
374     1496  
1338    1338  
1337    1337  
444     1332  
331     1324  
        ...   
72      72    
69      69    
65      65    
20      60    
57      57    
3       54    
26      52    
52      52    
51      51    
12      48    
48      48    
42      42    
37      37    
36      36    
35      35    
17      34    
34      34    
33      33    
32      32    
25      25    
23      23    
5       20    
6       18    
8       16    
15      15    
7       14    
13      13    
4       12    
11      11    
9       9     
Name: number_of_retweets, Length: 245, dtype: int64

In [ ]:
np.sum(clean_df.isnull().any(axis=1))

In [ ]:
clean_df.isnull().any(axis=0)

In [ ]:
tweets.iloc[clean_df[clean_df.isnull().any(axis=1)].index,:].head()

In [ ]:
clean_df.dropna(inplace=True)
clean_df.reset_index(drop=True,inplace=True)
clean_df.info()

In [ ]:
clean_df.head()

### Different approach to try solving that shifting that occured

In [ ]:
%%time
print("Cleaning and parsing the tweets...\n")
ct = pd.DataFrame(columns=cols)
for i in range(nums[0],nums[2]): 
    if( (i+1)%10000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, nums[2] ))                                                                  
    ct = ct.append({'tweet_text': tweet_cleaner(tweets['tweet_text'].iloc[i]), 'timestamp': tweets['timestamp'].iloc[i],
                                       'username': tweets['username'].iloc[i],'all_hashtags': tweets['all_hashtags'].iloc[i]}, ignore_index=True)

In [ ]:
ct.head()

In [ ]:
ct.describe()

In [ ]:
top_10_users = ct['username'].value_counts().head(10)
top_10_users

In [ ]:
top_10_tweets = ct['tweet_text'].value_counts().head(10)
top_10_tweets

In [ ]:
tweets_polarity = ct.copy()